In [1]:
import numpy as np
import cv2
import pandas as pd
import PIL
import utm
import matplotlib.pyplot as plt
import networkx as nx
import matplotlib as mpl
import os

In [2]:
def start_end_point(lat, lon):
    start = [lon - 55, lat + 50]
    end = [lon + 55, lat - 50]
    if start[0] > 180:
        start[0] = start[0] - 360
    elif start[0] < -180:
        start[0] = start[0] + 360
    if start[1] > 80:
        start[1] = start[1] - 160
    elif start[1] < - 80:
        start[1] = start[1] + 160

    if end[0] > 180:
        end[0] = end[0] - 360
    elif end[0] < -180:
        end[0] = end[0] + 360
    if end[1] > 80:
        end[1] = end[1] - 160
    elif end[1] < - 80:
        end[1] = end[1] + 160

    return(start, end)

def conv_func(data): # dont do this, ever, this is very bad. Will need to rewrite it for sure

    starting_points = []
    ending_points = []

    for i in range(len(data)):
        start, end = start_end_point(data.iloc[i]['latitude'], data.iloc[i]['longitude'])
        
        start_utm = utm.from_latlon(start[1], start[0])
        starting_points.append(start_utm[2:])
        
        end_utm = utm.from_latlon(end[1], end[0])
        ending_points.append(end_utm[2:])

    starting_zones = pd.DataFrame(starting_points)
    ending_zones = pd.DataFrame(ending_points)

    coords = pd.concat([data, starting_zones, ending_zones],axis=1)
    coords.columns = ['latitude', 'longitude', 's_zone', 's_band', 'e_zone', 'e_band']

    coords['ns_band'] = [ ord(x) - 64 for x in coords.s_band ]
    coords['ne_band'] = [ ord(x) - 64 for x in coords.e_band ]

    bins_zone = [i*15 for i in range(0,5)]
    labels_zone = [i for i in range(1,5)]

    bins_band = [i*6 for i in range(0,5)]
    labels_band = [i for i in range(1,5)]

    # Binning results
    coords['s_bin_zone'] = pd.DataFrame(pd.cut(coords['s_zone'], bins=bins_zone, labels=labels_zone))
    coords['s_bin_band'] = pd.DataFrame(pd.cut(coords['ns_band'], bins=bins_band, labels=labels_band))
    coords['e_bin_zone'] = pd.DataFrame(pd.cut(coords['e_zone'], bins=bins_zone, labels=labels_zone))
    coords['e_bin_band'] = pd.DataFrame(pd.cut(coords['ne_band'], bins=bins_band, labels=labels_band))

    return coords

In [ ]:
subject_list = [f for f in os.listdir('./data/head_tracking_data') if 'Subject' in f] # list of all subjects
video_list = [f for f in os.listdir('./data/video_files') if 'mp4' in f] # list of all videos


videos = {} # each video will contain all subjects

frames_per_segment = 30
video_id = video_list[19]
# for video_id in video_list[19]:
subjects = {} # for each subject, we store segments which are in seperate lists

for subject_id in subject_list:
    tmp_data = pd.read_fwf('data/head_tracking_data/'+ subject_id + '/' + video_id.split('.mp4')[0] + '.txt', header=None)
    tmp_data.columns = ['latitude', 'longitude']
    processed_data = conv_func(tmp_data)
    n_segments = int(len(tmp_data)/frames_per_segment) # ofc its not entirely coorect yet, it approximates how many segments\
    segments = []

    for seg in range(n_segments):
        # print('start ' , frames_per_segment * seg , ' end ', frames_per_segment * (seg+1))
        start = frames_per_segment * seg
        end = frames_per_segment * (seg+1)
        segments.append(processed_data[start:end])

    subjects[subject_id] = segments

videos[video_id] = subjects

In [ ]:
# need to figure out this still.
for subj in subject_list:
    for i in range(len(coords)-1):
        prev_top_left_zone = (coords.iloc[i]['s_bin_zone'], coords.iloc[i]['s_bin_band'])
        prev_bottom_right_zone = (coords.iloc[i]['e_bin_zone'], coords.iloc[i]['e_bin_band'])
        curr_top_left_zone = (coords.iloc[i+1]['s_bin_zone'], coords.iloc[i+1]['s_bin_band'])
        curr_bottom_right_zone = (coords.iloc[i+1]['e_bin_zone'], coords.iloc[i+1]['e_bin_band'])

        # connect previous set of [top left zone, bottom right zone] to current [top left zone, bottom right zone]
        G.add_edge((prev_top_left_zone, prev_bottom_right_zone), (curr_top_left_zone, curr_bottom_right_zone))
        
    videos['Help.mp4'][subj][3]